In [1]:
import numpy as np
import pandas as pd

In [34]:
from konlpy.tag import Okt
from konlpy.utils import pprint
from collections import Counter
from ckonlpy.tag import Postprocessor
from ckonlpy.tag import Twitter
from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora, models
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import codecs

/Users/simmigyeong/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [2]:
data1 = pd.read_excel("scrapingData/중고나라_휴대폰_2022-03-31_게시물.xlsx") 
data2 = pd.read_excel("scrapingData/중고나라_휴대폰_2_2022-03-31_게시물.xlsx") 
data3 = pd.read_excel("scrapingData/중고나라_휴대폰_3_2022-03-31_게시물.xlsx") 
data4 = pd.read_excel("scrapingData/중고나라_휴대폰_4_2022-03-31_게시물.xlsx") 
data5 = pd.read_excel("scrapingData/중고나라_휴대폰_5_2022-03-31_게시물.xlsx") 
data6 = pd.read_excel("scrapingData/중고나라_휴대폰_6_2022-03-31_게시물.xlsx") 
data7 = pd.read_excel("scrapingData/중고나라_휴대폰_7_2022-04-01_게시물.xlsx") 
data8 = pd.read_excel("scrapingData/중고나라_휴대폰_8_2022-04-01_게시물.xlsx")  

In [3]:
dataList = [data1, data2, data3, data4, data5, data6, data7, data8]
# data = pd.concat(dataList, ignore_index = True)
data = pd.concat(dataList)
data

,사이트,카테고리,제목,가격,상품 상태,작성 날짜,상품 설명,url,거래 상태
0,중고나라,휴대폰,아이폰12 프로 실버 128GB 90만 66682 수원,900000.0,사용감 있음,2022.03.31. 23:15,1.제품 >아이폰12 프로 실버 128GB 90만 66682 수원\n2.제품번호 >...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매(안전)
1,중고나라,휴대폰,[삼성 갤럭시S21] AA급 무잔상 바이올렛 256GB 48만원,480000.0,NaN,2022.03.31. 23:15,갤럭시S21 바이올렛 판매합니다~!\n☎☎ : 010.9671.3339\n기가수 :...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
2,중고나라,휴대폰,갤럭시S21 블랙색상 256용량 무잔상 깨끗한단말기 47만 판매합니다,470000.0,거의 새 것,2022.03.31. 23:15,갤럭시S21 블랙색상 256용량 판매합니다(통신사KT)\n폰상태 아래사진 그대로 깨...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
3,중고나라,휴대폰,갤럭시S20플러스 클라우드블루 256용량 깨끗한단말기 34만 판매합니다,340000.0,거의 새 것,2022.03.31. 23:15,갤럭시S20플러스 클라우드블루 256용량 판매합니다(통신사SKT)\n폰상태 아래사진...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
4,중고나라,휴대폰,아이폰XS 골드 64기가 배터리93% 23만원 판매합니다,230000.0,거의 새 것,2022.03.31. 23:15,아이폰XS 골드 64기가입니다 배터리 93%입니다 페이스X\n카메라 배터리 터치 통...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
...,...,...,...,...,...,...,...,...,...
371,중고나라,휴대폰,LG Q51 블루 무잔상 상태좋은폰 7만원판매해요,70000.0,거의 새 것,NaN,Q51 블루 무잔상 깨끗한단말기판매해요\n아래사진그대로 하자없이 깨끗한기기입니다\n...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
372,중고나라,휴대폰,V50S 블랙색상 256용량 기능정상 가성비 꿀매물 9만 판매합니다,90000.0,거의 새 것,2022.04.01. 01:51,V50S 블랙색상 256용량 판매합니다(통신사LGU+)\n폰상태 아래사진 그대로 깔...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
373,중고나라,휴대폰,V50S 블랙색상 256용량 기능정상 가성비 꿀매물 9만 판매합니다,90000.0,거의 새 것,2022.04.01. 01:51,V50S 블랙색상 256용량 판매합니다(통신사LGU+)\n폰상태 아래사진 그대로 깔...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
374,중고나라,휴대폰,V50S 블랙색상 256용량 기능정상 가성비 꿀매물 9만 판매합니다,90000.0,거의 새 것,2022.04.01. 01:51,V50S 블랙색상 256용량 판매합니다(통신사LGU+)\n폰상태 아래사진 그대로 깔...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매


In [4]:
data = data.drop_duplicates(['제목'])
data

,사이트,카테고리,제목,가격,상품 상태,작성 날짜,상품 설명,url,거래 상태
0,중고나라,휴대폰,아이폰12 프로 실버 128GB 90만 66682 수원,900000.0,사용감 있음,2022.03.31. 23:15,1.제품 >아이폰12 프로 실버 128GB 90만 66682 수원\n2.제품번호 >...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매(안전)
1,중고나라,휴대폰,[삼성 갤럭시S21] AA급 무잔상 바이올렛 256GB 48만원,480000.0,NaN,2022.03.31. 23:15,갤럭시S21 바이올렛 판매합니다~!\n☎☎ : 010.9671.3339\n기가수 :...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
2,중고나라,휴대폰,갤럭시S21 블랙색상 256용량 무잔상 깨끗한단말기 47만 판매합니다,470000.0,거의 새 것,2022.03.31. 23:15,갤럭시S21 블랙색상 256용량 판매합니다(통신사KT)\n폰상태 아래사진 그대로 깨...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
3,중고나라,휴대폰,갤럭시S20플러스 클라우드블루 256용량 깨끗한단말기 34만 판매합니다,340000.0,거의 새 것,2022.03.31. 23:15,갤럭시S20플러스 클라우드블루 256용량 판매합니다(통신사SKT)\n폰상태 아래사진...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
4,중고나라,휴대폰,아이폰XS 골드 64기가 배터리93% 23만원 판매합니다,230000.0,거의 새 것,2022.03.31. 23:15,아이폰XS 골드 64기가입니다 배터리 93%입니다 페이스X\n카메라 배터리 터치 통...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
...,...,...,...,...,...,...,...,...,...
364,중고나라,휴대폰,아이폰11프로 64기가 그린 상태좋고 깨끗한 단말기 46만원 판매해요,460000.0,거의 새 것,2022.04.01. 01:51,아이폰11프로 64기가 그린 상태좋고 깨끗한 단말기 46만원 판매해요\n선택약정할인...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
366,중고나라,휴대폰,갤럭시Z플립 퍼플색상 256용량 깨끗한단말기 저렴하게 22만 판매합니다,220000.0,거의 새 것,2022.04.01. 01:51,갤럭시Z플립 퍼플색상 256용량 판매합니다(통신사KT)\n폰상태 아래사진 그대로 깨...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
369,중고나라,휴대폰,아이폰SE2 블랙 64G 매우깨끗한기기 19만원판매합니다!,190000.0,거의 새 것,2022.04.01. 01:51,아이폰SE2 블랙색상 64기가 판매합니다\n외관상태 아래사진그대로 매우깨끗합니다!\...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
370,중고나라,휴대폰,6개월 보증] 노트8 골드 64G A급 15만원 사은품포함/83106,150000.0,거의 새 것,2022.04.01. 01:51,제품정보\n＠ 모델명 : 노트8 / 64G \n＠ 색상: 골드\n＠ 판매가격 : 1...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매(안전)


In [5]:
data.isnull().sum()

사이트        0
카테고리       0
제목         1
가격         1
상품 상태    138
작성 날짜      2
상품 설명      2
url        1
거래 상태      1
dtype: int64

In [6]:
data = data.dropna(subset=['제목'])
data

,사이트,카테고리,제목,가격,상품 상태,작성 날짜,상품 설명,url,거래 상태
0,중고나라,휴대폰,아이폰12 프로 실버 128GB 90만 66682 수원,900000.0,사용감 있음,2022.03.31. 23:15,1.제품 >아이폰12 프로 실버 128GB 90만 66682 수원\n2.제품번호 >...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매(안전)
1,중고나라,휴대폰,[삼성 갤럭시S21] AA급 무잔상 바이올렛 256GB 48만원,480000.0,NaN,2022.03.31. 23:15,갤럭시S21 바이올렛 판매합니다~!\n☎☎ : 010.9671.3339\n기가수 :...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
2,중고나라,휴대폰,갤럭시S21 블랙색상 256용량 무잔상 깨끗한단말기 47만 판매합니다,470000.0,거의 새 것,2022.03.31. 23:15,갤럭시S21 블랙색상 256용량 판매합니다(통신사KT)\n폰상태 아래사진 그대로 깨...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
3,중고나라,휴대폰,갤럭시S20플러스 클라우드블루 256용량 깨끗한단말기 34만 판매합니다,340000.0,거의 새 것,2022.03.31. 23:15,갤럭시S20플러스 클라우드블루 256용량 판매합니다(통신사SKT)\n폰상태 아래사진...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
4,중고나라,휴대폰,아이폰XS 골드 64기가 배터리93% 23만원 판매합니다,230000.0,거의 새 것,2022.03.31. 23:15,아이폰XS 골드 64기가입니다 배터리 93%입니다 페이스X\n카메라 배터리 터치 통...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
...,...,...,...,...,...,...,...,...,...
364,중고나라,휴대폰,아이폰11프로 64기가 그린 상태좋고 깨끗한 단말기 46만원 판매해요,460000.0,거의 새 것,2022.04.01. 01:51,아이폰11프로 64기가 그린 상태좋고 깨끗한 단말기 46만원 판매해요\n선택약정할인...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
366,중고나라,휴대폰,갤럭시Z플립 퍼플색상 256용량 깨끗한단말기 저렴하게 22만 판매합니다,220000.0,거의 새 것,2022.04.01. 01:51,갤럭시Z플립 퍼플색상 256용량 판매합니다(통신사KT)\n폰상태 아래사진 그대로 깨...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
369,중고나라,휴대폰,아이폰SE2 블랙 64G 매우깨끗한기기 19만원판매합니다!,190000.0,거의 새 것,2022.04.01. 01:51,아이폰SE2 블랙색상 64기가 판매합니다\n외관상태 아래사진그대로 매우깨끗합니다!\...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
370,중고나라,휴대폰,6개월 보증] 노트8 골드 64G A급 15만원 사은품포함/83106,150000.0,거의 새 것,2022.04.01. 01:51,제품정보\n＠ 모델명 : 노트8 / 64G \n＠ 색상: 골드\n＠ 판매가격 : 1...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매(안전)


In [7]:
data[data['작성 날짜'].isnull()]

,사이트,카테고리,제목,가격,상품 상태,작성 날짜,상품 설명,url,거래 상태
371,중고나라,휴대폰,LG Q51 블루 무잔상 상태좋은폰 7만원판매해요,70000.0,거의 새 것,NaN,Q51 블루 무잔상 깨끗한단말기판매해요\n아래사진그대로 하자없이 깨끗한기기입니다\n...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매


In [8]:
data[data['상품 설명'].isnull()]

,사이트,카테고리,제목,가격,상품 상태,작성 날짜,상품 설명,url,거래 상태
131,중고나라,휴대폰,갤럭시Z플립 256GB 정상해지 무잔상 액정깨끗 31만,310000.0,NaN,2022.03.31. 23:23,NaN,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1182 entries, 0 to 371
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   사이트     1182 non-null   object 
 1   카테고리    1182 non-null   object 
 2   제목      1182 non-null   object 
 3   가격      1182 non-null   float64
 4   상품 상태   1045 non-null   object 
 5   작성 날짜   1181 non-null   object 
 6   상품 설명   1181 non-null   object 
 7   url     1182 non-null   object 
 8   거래 상태   1182 non-null   object 
dtypes: float64(1), object(8)
memory usage: 92.3+ KB


In [10]:
dataTitle = pd.DataFrame(data['제목'])
dataTitle

,제목
0,아이폰12 프로 실버 128GB 90만 66682 수원
1,[삼성 갤럭시S21] AA급 무잔상 바이올렛 256GB 48만원
2,갤럭시S21 블랙색상 256용량 무잔상 깨끗한단말기 47만 판매합니다
3,갤럭시S20플러스 클라우드블루 256용량 깨끗한단말기 34만 판매합니다
4,아이폰XS 골드 64기가 배터리93% 23만원 판매합니다
...,...
364,아이폰11프로 64기가 그린 상태좋고 깨끗한 단말기 46만원 판매해요
366,갤럭시Z플립 퍼플색상 256용량 깨끗한단말기 저렴하게 22만 판매합니다
369,아이폰SE2 블랙 64G 매우깨끗한기기 19만원판매합니다!
370,6개월 보증] 노트8 골드 64G A급 15만원 사은품포함/83106


In [11]:
data[data['제목'].str.contains('LG')]

,사이트,카테고리,제목,가격,상품 상태,작성 날짜,상품 설명,url,거래 상태
13,중고나라,휴대폰,LG 보증] Q730 64G 블랙 S급 10만원 사은품포함/63858,100000.0,거의 새 것,2022.03.31. 23:14,제품정보\n＠ 모델명 : Q730 / 64G \n＠ 색상: 블랙\n＠ 판매가격 : ...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매(안전)
22,중고나라,휴대폰,LG V40 128기가 블루 액정파손 기능정상 무잔상 6만원 판매해요,60000.0,사용감 있음,2022.03.31. 23:14,LG V40 128기가 블루 액정파손 기능정상 무잔상 6만원 판매해요\n선택약정할인...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
35,중고나라,휴대폰,LG V50S ThinQ (V510) 256GB 블랙 무잔상 16만원,160000.0,NaN,2022.03.31. 23:13,안녕하세요. LG V50S ThinQ (V510) 256GB 블랙 판매합니다\n최초...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
37,중고나라,휴대폰,LG Q720 (Q725) A급 전국 최저가 팝니다. 5만원 판매!!!,50000.0,거의 새 것,2022.03.31. 23:13,★ 제품정보 ★\nLG Q720 (Q725) A급 전국 최저가 팝니다. 5만원 판매...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
112,중고나라,휴대폰,LG V30 레드 초깔끔 8.8만원 판매해요(64G),88000.0,거의 새 것,2022.03.31. 23:06,LG V30 레드 초깔끔 8.8만원 판매해요(64G)\n최초 통신사 LG 입니다만 ...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
...,...,...,...,...,...,...,...,...,...
295,중고나라,휴대폰,LG Q7 플러스 64기가 블루 S급 무잔상폰 5만원판매해요,50000.0,거의 새 것,2022.04.01. 01:57,Q7플러스 64기가 블루 S급 무잔상폰 판매해요\n아래사진그대로 하자없이 상태 매우...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
297,중고나라,휴대폰,LG Q92 블랙 128G 20년11월개통 무잔상A급 10만원판매합니다!,100000.0,거의 새 것,2022.04.01. 01:57,LG Q92 블랙색상 128G 판매합니다\n외관상태 아래사진과같이 매우깨끗합니다!\...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
303,중고나라,휴대폰,LG G7 라즈베리 64G 8만원에 팔아요,80000.0,사용감 있음,2022.04.01. 01:57,"LG G7 라즈베리 64G 판매합니다\n통신사 구분없이 3사 호환가능합니다 , 25...",https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
331,중고나라,휴대폰,LG V50 128GB 외관 깔끔 저렴 정상해지 10만~,100000.0,사용감 있음,2022.04.01. 01:54,LG V50 128GB 외관 깔끔 저렴 정상해지 10만~\n1번 -[KT]LG V5...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매


In [12]:
dataTitle.query('제목.str.contains("아이폰")', engine='python')

,제목
0,아이폰12 프로 실버 128GB 90만 66682 수원
4,아이폰XS 골드 64기가 배터리93% 23만원 판매합니다
7,[애플 아이폰SE] 귀한 매물 64GB 핑크 13만원
9,"아이폰13 PRO MAX 프로 맥스 자급제미개봉,당일개통,박스폰삽니다"
14,아이폰11 화이트 128G 배터리(88%)
...,...
336,아이폰11프로 블랙 256GB 60만 72148 /수원
351,아이폰xs 256G 블랙 성능 77% 기능정상 팝니다
356,아이폰7플러스 매트블랙 128GB 23만 02020 /수원
364,아이폰11프로 64기가 그린 상태좋고 깨끗한 단말기 46만원 판매해요


In [13]:
data[data['제목'].str.contains('갤럭시')]

,사이트,카테고리,제목,가격,상품 상태,작성 날짜,상품 설명,url,거래 상태
1,중고나라,휴대폰,[삼성 갤럭시S21] AA급 무잔상 바이올렛 256GB 48만원,480000.0,NaN,2022.03.31. 23:15,갤럭시S21 바이올렛 판매합니다~!\n☎☎ : 010.9671.3339\n기가수 :...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
2,중고나라,휴대폰,갤럭시S21 블랙색상 256용량 무잔상 깨끗한단말기 47만 판매합니다,470000.0,거의 새 것,2022.03.31. 23:15,갤럭시S21 블랙색상 256용량 판매합니다(통신사KT)\n폰상태 아래사진 그대로 깨...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
3,중고나라,휴대폰,갤럭시S20플러스 클라우드블루 256용량 깨끗한단말기 34만 판매합니다,340000.0,거의 새 것,2022.03.31. 23:15,갤럭시S20플러스 클라우드블루 256용량 판매합니다(통신사SKT)\n폰상태 아래사진...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
5,중고나라,휴대폰,갤럭시S20 그레이 128기가 S급 정상해지 공기기+박스 팝니다,350000.0,거의 새 것,2022.03.31. 23:15,"100% 사진과 동일한 제품\n정상해지 공기기로 선택약정할인,3사유심,알뜰유심,선불...",https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
6,중고나라,휴대폰,(단말자급제)갤럭시폴드2 256기가 브론즈 상태좋은폰 77만원 판매해요,770000.0,사용감 있음,2022.03.31. 23:15,(단말자급제)갤럭시폴드2 256기가 브론즈 상태좋은폰 77만원 판매해요\n선택약정할...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
...,...,...,...,...,...,...,...,...,...
352,중고나라,휴대폰,갤럭시노트8 최상품판매 정상판매중[부천][송내역/중고폰],110000.0,거의 새 것,2022.04.01. 01:52,"👆 위 중고나라 앱 다운로드 링크를 통해, 앱 다운로드하여 판매 및 구매 게시글을 ...",https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
353,중고나라,휴대폰,갤럭시S20 128G 그레이(G981) 무잔상 30만원,300000.0,NaN,2022.04.01. 01:52,제품정보\n제품 : 갤럭시S20 128G (에스이십 G981)\n개통 : SKT 2...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매(안전)
360,중고나라,휴대폰,자급제 갤럭시폴드2 브론즈색상 256용량 무잔상 깔끔한단말기 75만 팝니다,750000.0,거의 새 것,2022.04.01. 01:52,갤럭시폴드2 브론즈색상 256용량 판매합니다(단말자급제)\n폰상태 아래사진 그대로 ...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매
361,중고나라,휴대폰,갤럭시 노트9 퍼플 128G 가성비꿀매물 15만원팝니다,150000.0,사용감 있음,2022.04.01. 01:52,갤럭시 노트9 퍼플 128G 가성비꿀매물 15만원팝니다\n선택약정할인25% 가능 단...,https://cafe.naver.com/ca-fe/cafes/10050146/ar...,판매


In [31]:
from ckonlpy.utils import load_wordset
passwords = load_wordset('postprocess/passwords.txt')
stopwords = load_wordset('postprocess/stopwords.txt')

from ckonlpy.utils import load_replace_wordpair
replace = load_replace_wordpair('postprocess/replace.txt')

from ckonlpy.utils import load_ngram
ngrams = load_ngram('postprocess/ngrams.txt')

/Users/simmigyeong/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
Okt = Okt()
twitter = Twitter()

/Users/simmigyeong/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


SystemError: java.lang.ClassNotFoundException: org.jpype.classloader.DynamicClassLoader